<table width="100%">
    <tr style="border-bottom:solid 2pt #009EE3">
        <td style="text-align:left" width="10%">
            <a href="FILENAME" download><img src="../../images/icons/download.png"></a>
        </td>
        <td style="text-align:left" width="10%">
            <a href="SOURCE" target="_blank"><img src="../../images/icons/program.png" title="Be creative and test your solutions !"></a>
        </td>
        <td></td>
        <td style="text-align:left" width="5%">
            <a href="../MainFiles/biosignalsnotebooks.ipynb"><img src="../../images/icons/home.png"></a>
        </td>
        <td style="text-align:left" width="5%">
            <a href="../MainFiles/contacts.ipynb"><img src="../../images/icons/contacts.png"></a>
        </td>
        <td style="text-align:left" width="5%">
            <a href="https://github.com/biosignalsnotebooks/biosignalsnotebooks" target="_blank"><img src="../../images/icons/github.png"></a>
        </td>
        <td style="border-left:solid 2pt #009EE3" width="15%">
            <img src="../../images/ost_logo.png">
        </td>
    </tr>
</table>

<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_6"><div id="image_img"
        class="header_image_6"></div></td>
        <td class="header_text"> Calculate Time of Flight </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">extract&#9729;pre-process&#9729;accelerometer&#9729;force platform</td>
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

Jumping is an <span class="color1"><a href=https://ojs.ub.uni-konstanz.de/cpa/article/view/2325>important task<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span> to assess the physical health of individuals, particularly in athletes. There are many features that may be studied in each jump, such as the time of flight or loft. This particular feature may indicate the muscle strength of the lower body in relation to the weight of the athlete and can be used as an assessment for the training plan of the said athlete.

The current <span class="color4"><strong>Jupyter Notebook</strong></span> focuses on the presentation of a method to extract the loft time using either an <strong><a href=https://plux.info/sensors/264-accelerometer-acc-820201205.html>acceleration signal<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></strong> or a <strong><a href=https://plux.info/sensors/288-force-platform-820202504.html>force signal<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></strong> of different sensors. 

<hr>

<p class="steps">1 - Acquisition Procedure</p>
<strong><span class="color1">In order to assess the loft time and extract the multiple features from the accelerometer and force platform, we first need to define a replicable protocol.</span></strong>
<br>In this case, we taped the accelerometer to the lower back of the subject in order to be as close as possible to the center of mass. Thus, we can track the exact accelerometry of the subject and have a higher degree of certainty regarding their movements. Furthermore, the subjects jumped vertically on the force platform without any horizontal movement.

<table style="width:12%">
  <tr>
    <td><img src="../../images/extract/loft_time/loft_time_gui.gif" height="326" controls></td>
  </tr>
</table>

<p class="steps">2 - Load Signals</p>
<strong><span class="color1">In this section we will show how to load the signals using the biosignalsnotebooks Python package.</span></strong>

<p class="steps">2.1 - Import the required packages</p>

In [1]:
import biosignalsnotebooks as bsnb
import numpy as np

<p class="steps">2.2 - Load the signals</p>
Given that the acquisitions involved two devices - the force platform and a <strong><span class="color2">biosignalsplux</span></strong> accelerometer - we need to load the signals from both of them. Fortunately, the signals are stored in a single file, therefore, we only need to load it once. After that, we need to extract the signals of each device from the file given the <i>mac address</i>.

In [2]:
# Define the path of the file that has the signals of each device
path_subject_1 = "../../signal_samples/loft_time_subject_1.h5"

# Load the data from the file
data_1 = bsnb.load(path_subject_1)

# Identify the mac addresses of each device
force_platform_1, accelerometer_1 = data_1.keys()

# Get the signals acquired by the force_platform
force_platform_signals_1 = data_1[force_platform_1]

# Get the signals acquired by the accelerometer
accelerometer_signals_1 = data_1[accelerometer_1]

Each device acquired a different number of channels, corresponding to different signals. 
<br>Specifically, the force platform acquired data using four channels, while the accelerometer acquired data on three channels corresponding to each tridimensional axis.

<strong><span class="color1">The next cells will show how to get the data from each device and from each channel of each device.</span></strong>
<br> For that, we will first initiate different lists for different devices and different experiments. Then, we will iterate over each channel of each device and store those signals in the corresponding list.

<p class="steps">2.3 - Initialization of lists that will store the physiological data contained inside the loaded file</p>

In [3]:
# These are the lists where we will store the signals from each channels of the force platform of each experiment.
# They are initiated as empty lists so we can append items to them.
force_platform_channels_1 = []

# And these are the lists where we will store the signals from each direction of the accelerometer of each experiment.
accelerometer_direction_1 = []

<p class="steps">2.4 - Populate the initialized lists</p>
This is the part where we get the signal from each channel and append it to the corresponding list.
<br>The syntax of the following code lines may be a bit odd at first sight, so we recommend you to read: <a href="https://realpython.com/iterate-through-dictionary-python/">https://realpython.com/iterate-through-dictionary-python/</a>

<strong><span class="color2">Force Platform</span></strong>

In [4]:
# Basically, force_platform_signals_1.keys() is a strutucture that iterates over the channels of the force platform device.
for channel in force_platform_signals_1.keys():
    # First, we get the signal of the channel
    channel_signal_1 = force_platform_signals_1[channel]
    # Then append the signal to the list that will contain the signals.
    force_platform_channels_1.append(channel_signal_1)

<strong><span class="color7">Accelerometer</span></strong>

In [5]:
# Now we need to do the same for the accelerometer. Note that the code is very similar compared to the force platform.
for direction in accelerometer_signals_1.keys():
    direction_signal_1 = accelerometer_signals_1[direction]
    accelerometer_direction_1.append(direction_signal_1)

<table width="100%">
    <tr>
        <td width="40%" style="text-align:center">
            <hr>
        </td>
        <td width="10%" style="text-align:center">
            CHECKPOINT
        </td>
        <td width="40%" style="text-align:center">
            <hr>
        </td>
    </tr>
</table>

The next plots are intended to show the signals generated by the force platform. 

Here, they were trimmed to show only the part that interests us, which is the jump, and we can clearly see that it has a remarkable shape: first the signals are stable, meaning that the subject is at rest. Then, when the subjects start to flex their legs, the signal increases, meaning that there is a rising force being applied to the platform. 

The minimum force applied during the flex corresponds to the position of maximum flex of the legs, in which the force on the platform is reduced. Then, there is a significant increase of the signal that corresponds to the increasing force being applied as the legs start to stretch to lift the whole body. The next segment of the signal, corresponding of 0's, is the time interval when the feet leave the platform and, so, there is no pressure on it. The next structure, composed of a peak and a valley corresponds to the landing part of the jump.

<strong><span class="color7">Note:</span></strong> Of course this is an estimate, because when no force is being applied, the platform still has to go back to the baseline position, in which it still detects some force, creating a bias. One way to take that into consideration would be to increase the threshold that flags the "no force" state, when no force is being applied to the platform.

In [6]:
fs = 500
time = bsnb.generate_time(force_platform_signals_1['CH1'], fs)

begin_1 = 19*fs
end_1 = 23*fs

times_1 = []
force_platform_channels_1 = []
for key in force_platform_signals_1.keys():
    times_1.append(time[begin_1:end_1])
    force_platform_channels_1.append(force_platform_signals_1[key][begin_1:end_1])

tr_1 = np.min(force_platform_channels_1[0]) + np.ptp(force_platform_channels_1[0])*.01
index_1 = np.where(force_platform_channels_1[0] < tr_1)[0]
index_1 = (np.array([index_1[a] for a in [0, -1]]) + begin_1)/fs

bsnb.plot(times_1, force_platform_channels_1, grid_plot=True, grid_lines=2, grid_columns=2, x_axis_label='Time (s)', 
          vert_lines=[index_1, index_1, index_1, index_1], y_axis_label='Raw Data',
          title=["Channel 1", "Channel 2", "Channel 3", "Channel 4"])

For the accelerometer, the signals are not as clear as the ones generated by the force platform, but may still be valid to assess the loft time. 

Essentially, we can look at each axis to see the differences between the rest state and the jump. For example, on the <strong><span class="color2">x axis</span></strong>, we see an increase of the variance of the acceleration during the jump; in the <strong><span class="color5">y axis</span></strong> we see a similar structure as the one seen in the force platform; in the <strong><span class="color7">z axis</span></strong> the structure seems a mix of the x and y axis. 

The problem with this analysis, is that it depends on the placement of the accelerometer, therefore, we should be able to do the analysis regardless of the said placement. In order to do that, we calculated the magnitude of the signal, which corresponds to the combination of the 3 axis to generate one signal that is equally influenced by each signal, as expressed in the next expression:

<div align=center> $ magnitude = \sqrt{x^2 + y^2 + z^2} $ </div>

In the calculus of the squared value of each direction, we subtract the mean and add the minimum value of all axis in order to normalize the signals. This way, we guarantee that no axis has more influence than the rest due to a baseline acceleration, such as, gravity. We also guarantee that all signals are positive.

<table width="100%">
    <tr>
        <td width="40%" style="text-align:center">
            <hr>
        </td>
        <td width="10%" style="text-align:center">
            END OF CHECKPOINT
        </td>
        <td width="40%" style="text-align:center">
            <hr>
        </td>
    </tr>
</table>

In [7]:
# These first lines will get each direction that are required to calculate the magnitude.
experiment_1_direction_x = np.array(accelerometer_direction_1[0])
experiment_1_direction_y = np.array(accelerometer_direction_1[1])
experiment_1_direction_z = np.array(accelerometer_direction_1[2])

# Now, we need to calculate the square of each normalised direction
experiment_1_direction_x_squared = (experiment_1_direction_x - np.mean(experiment_1_direction_x) + np.min([accelerometer_direction_1]))**2
experiment_1_direction_y_squared = (experiment_1_direction_y - np.mean(experiment_1_direction_y) + np.min([accelerometer_direction_1]))**2
experiment_1_direction_z_squared = (experiment_1_direction_z - np.mean(experiment_1_direction_z) + np.min([accelerometer_direction_1]))**2

# The next step is to sum them
sum_all_direction_squared_1 = experiment_1_direction_x_squared + experiment_1_direction_y_squared + experiment_1_direction_z_squared

# And finaly apply the square root
magnitude_1 = np.sqrt(sum_all_direction_squared_1)


# Given the step by step explanation, we will now show how to do the same in a shorter way for the second experiment.

# Iterate over all directions and add the noramlised squared value of them to a variable
all_direction_squared_2 = []
for direction in accelerometer_direction_2:
    experiment_2_direction_squared = (np.array(direction) - np.mean(direction) + np.min(accelerometer_direction_2))**2
    all_direction_squared_2.append(experiment_2_direction_squared)

# Sum the square of each direction
sum_all_direction_squared_2 = np.sum(all_direction_squared_2, axis=0)
    
# Calculate the square root operation to the sum of the square of each direction
magnitude_2 = np.sqrt(sum_all_direction_squared_2)

NameError: name 'accelerometer_direction_2' is not defined

On the magnitude signal, we see the gathered influence of each axis that does not depend on the orientation of the sensor while acquiring data.

<strong><span class="color2">&gt;&gt;&gt;&gt;&gt;&gt; Subject 1</span></strong>

In [ ]:
accelerometer_direction_1 = []
for channel in accelerometer_signals_1.keys():
    accelerometer_direction_1.append(accelerometer_signals_1[channel][begin_1:end_1])

magnitude_1 = magnitude_1[begin_1:end_1]
accelerometer_direction_1.append(magnitude_1)

bsnb.plot(times_1, accelerometer_direction_1, grid_plot=True, grid_lines=2, grid_columns=2, x_axis_label='Time (s)', 
          vert_lines=[index_1, index_1, index_1, index_1],
          y_axis_label='Raw Data', title=["x Axis", "y Axis", "z Axis", "Magnitude of the Axis"])

<strong><span class="color7">&gt;&gt;&gt;&gt;&gt;&gt; Subject 2</span></strong>

In [ ]:
accelerometer_direction_2 = []

for key in accelerometer_signals_2.keys():
    accelerometer_direction_2.append(accelerometer_signals_2[key][begin_2:end_2])

magnitude_2 = magnitude_2[begin_2:end_2]
accelerometer_direction_2.append(magnitude_2)

bsnb.plot(times_2, accelerometer_direction_2, grid_plot=True, grid_lines=2, grid_columns=2, x_axis_label='Time (s)', 
          vert_lines=[index_2, index_2, index_2, index_2], y_axis_label='Raw Data',
          title=["x Axis", "y Axis", "z Axis", "Magnitude of the Axis"])

The previous statement is valid for both experiments. The reason to include two experiments instead of only one, in this <span class="color4"><strong>Jupyter Notebook</strong></span>, is related with the fact that, even though the protocol was the same for both, the signals acquired for the different experiments are very different. 

Specifically, on the first experiment the flight time corresponds to a valley on the magnitude signal while in the second case, the flight time is represented by a rise and quasi-plateau on the signal that then decreases again. Thus, our approach has to be generic enough to have both behaviors into account.

<p class="steps">3 - Signal Analysis</p>
<strong><span class="color1">Following the eye analysis and structures identification, we can start to computationally identify the structures that indicate the beginning and ending of the jump.</span></strong>
<br>Specifically, though the difference between the signals, we saw that the flight time corresponds to a plateau in each signal - one of them is a positive plateau and the other one is a negative plateau.

The loft time corresponds to the time the subject is not touching the platform, which is identifiable as the plateau between the peaks that correspond to the jumping and landing. 
<br>In the case of the <strong>first experiment</strong>, the loft time is estimated to be <strong>0.340 seconds</strong> while in the <strong>second experiment</strong> the loft time is estimated to be <strong>0.404 seconds</strong> (refer to the figures of the previous section). 

In this section, we will explain how to implement an algorithm that may be able to calculate those times using the accelerometer signal.

<p class="steps">3.1 - Algorithm Description</p>

The algorithm is based on the structure of the plateaus. The plateaus, shown in the figures below will help us to analyze the general case.

<table style="width:100%">
  <tr>
    <td>
        <figure>
          <img src="../../images/extract/loft_time/positive_plateau.png" height="1000">
          <figcaption style="text-align: center">Positive Plateau.</figcaption>
        </figure>
      </td>
      <td>
        <figure>
          <img src="../../images/extract/loft_time/negative_plateau.png" height="1000">
          <figcaption style="text-align: center">Negative Plateau.</figcaption>
        </figure>
      </td>
  </tr>
</table>

<span class="color4"><strong>Positive Plateau</strong></span>
<br>We will analyze each specific point and all the structures around them.
<ul>
  <li>On the left side of point number 1 there is a high positive slope, meaning that it has a high and positive derivate while on the right side the signal is constant, meaning the derivate is 0 or very low.</li>
  <li>On number 2 there is only one condition, which is that the signal is always constant, so, the derivate is 0 or very low.</li>
  <li>On the left of point number 3 the signal is constant and, therefore, the derivate is 0 or very low. On the right side of the point there is a high slope and the signal decreases, meaning that the derivate is high and negative.</li>
</ul>

<span class="color13"><strong>Negative Plateau</strong></span>
<br>The analysis is very similar to the positive plateau with some nuances, which are what makes the difference.
<ul>
  <li>On the left side of point number 1 there is a high negative slope, meaning that it has a high and negative derivate while on the right side the signal is constant, meaning the derivate is 0 or very low.</li>
  <li>On number 2 the condition is the same as on the previous case, which is that the signal is constant and, thus, the derivate must be 0 or very low.</li>
  <li>On the left of point number 3 the signal is constant and, therefore, the derivate is 0 or very low. On the right side of the point there is a high slope that rises the signal, meaning that the derivate is high and positive.</li>
</ul>

The next task consists on translating the previous analysis into code. 
<br>What we have to take into account is that when we refer to <i>'very low'</i> or <i>'very high'</i> terms, we need to specify thresholds that objectively define what those mean.

<p class="steps">3.1.1 - First Experiment</p>
Now, we will start by the first experiment, which has a negative plateau. 
<br>First, we will start by defining all the thresholds that we will need to apply. These values can be adapted to each individual application, but in ours we will define them based on empirical observations. Specifically, based on our observations, both very low and very high threshold values will be the same and the same values will be used for the positive threshold.

In [ ]:
# Thresholds for the limiting points (1 and 3)
very_low_limiting_threshold = 100
very_high_limiting_threshold = 100

# Threshold for the point number 2
very_low_threshold = 700

# Sampling frequency of the acquisitions
fs = 500 # Hz

# Threshold specifying that there are no jumps that take less than 0.1 seconds. 
# The multiplication by the sampling rate is the conversion to the number of points in 0.1 seconds.
delta_t = 0.1*fs

Next, we will need to iterate over the signal and apply the conditions of each of the three points. Given that in our analyses is based on the signal <strong>and</strong> the derivate, we need to first calculate it. Moreover, we will apply a <a href="https://plux.info/signal-processing/453-digital-filtering-pre-process-step.html"> low pass filter<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> in order to reduce the noise and keep only the most relevant structures.

In [ ]:
# Application of the low pass filter using a cut-off frequency of 10 Hz. The use_filtfilt argument states that
# the same filter is applied forward and backwards in order to reduce the artifacts the first filter may introduce.
filtered_magnitude_1 = bsnb.lowpass(magnitude_1, 10, fs=fs, use_filtfilt=True)

# Here we get the derivate of the signal of magnitude of the first experiment
derivate_magnitude_1 = np.diff(filtered_magnitude_1)

<strong><span class="color7">Now we have all the conditions to write the actual code that will look for each point. We have comments on the code in order to facilitate the explanation, so, beware of them while reading the code.</span></strong>

First, we will initialize the list where we will store the limiting points. Specifically, we will use one list to store the provisory points and another that will store the final points. The difference is that the final list will contain the points that meet all conditions to be considered a limiting point.

In [ ]:
# Here we initialise the list that will contain the provisory points limiting the plateau(s).
low_plateau = []

# Here we initialise the list that will contain the final points limiting the plateau(s).
low_final_plateau = []

Second, we initialize a flag that will inform the algorithm what was the point that was last identified. 

This is important because we should not look for consecutive left point or consecutive right points. The order should be always the same: we first find a left limiting point, then we find a right limiting point. Additionally, we can start look again for left limiting point only after the both limiting points were found.

In [ ]:
# This flag will be used to tell us if the algorithm has found a left limiting point, because it does not make sense to look for consecutive
# left limiting points if we have not found a right one.
low_flag = False

This is the tricky part. The next cell shows how we look for the provisory limiting points. Specifically, it iterates over the signal and checks if the conditions we set are met.

In [ ]:
# Iterate over all points in the derivate except the first and last ones. This is because we will be looking for left and right points, and, so,
# we cannot look to the left of the first point and the right of the last point.
for point in range(1, len(derivate_magnitude_1)-1):
    
    # Get the point immediately at the left of the current one.
    left_point = derivate_magnitude_1[point-1]
    
    # Get the point immediately at the right of the current one.
    right_point = derivate_magnitude_1[point+1]
    
    # This says that only if the flag is false, meaning that no left limiting point was found, does the algorithm look for a left limiting point.
    # Else, it will look for a right limiting point.
    if not low_flag:
        # Definition of point number 1
        
        # If all the identified conditions verify, we store the point as a left limiting point.
        if np.abs(left_point) > very_high_limiting_threshold and  left_point < 0 and np.abs(right_point) < very_low_limiting_threshold:
            low_plateau.append(point)
            
            # The flag turns to true to specify that we have found a left limiting point.
            low_flag = True
    else:
        # Definition of point number 3
        # We check if the time threshold specified before is respected and only if it is does the algorithm look for a right limiting point.
        if point - low_plateau[-1] > delta_t:
             
            # If all the identified conditions verify, we store the point as a right limiting point.
            if np.abs(left_point) < very_low_limiting_threshold and np.abs(right_point) > very_high_limiting_threshold and right_point > 0:
                low_plateau.append(point)
                
                # Then we say that the flag is false so that new left limiting points can be found.
                low_flag = False

Given the provisory points, we should check if there is any left limiting point without a right correspondent. Due to the construction of the code, it can only happen if the last point is a left point and, so, we just need to see the state of the flag. If it is True, it means that the last point it found was a left one, and thus can be eliminated from the list.

In [ ]:
# If a left limiting point was found but not the right corresponding one, we should discard it.
if low_flag:
    low_plateau.pop()

Finally, we can check if the final condition is met. The final condition is that there is no point in between limiting points with a derivate higher than the threshold we defined. Only if that criteria is met are the points considered as true limiting points and are stored on the list containing the final points.

In [ ]:
# Only if we have more than two points do we check if they are suitable to be considered as limits of a plateau.
if len(low_plateau) >= 2:
    
    # Here we will iterate over all points but in a pairwise condition, because a plateau is always limited by two points in our algorithm.
    for point in range(0, len(low_plateau)-2, 2):
        
        # Definition of point number 2
        
        # If there are no point in between limiting points with a derivate higher than the threshold, then those are really limiting points and
        # are stored in the final list.
        if not np.any(derivate_magnitude_1[low_plateau[point]:low_plateau[point+1]] > very_low_threshold):
            low_final_plateau.append(low_plateau[point])
            low_final_plateau.append(low_plateau[point+1])

Now, we will check if the algorithm worked as expected and compare the estimated time using this algorithm to the time we calculated using the force platform signals. 

The plot below shows the points the algorithm found as the vertical lines and we see that the plateau was found.

In [ ]:
time_1 = times_1[0]

start_1 = (np.array(low_plateau) + begin_1)/fs

print(f"The estimated time is {np.diff(low_plateau)[0]/fs} which differs from the ground truth by {np.abs(0.340 - np.diff(low_plateau)[0]/fs):.3}.")

bsnb.plot(times_1[0], filtered_magnitude_1, vert_lines=start_1)

<p class="steps">3.1.2 - Second Experiment</p>
Given that we have specified all the parameters required to detect the negative plateau and that we will use the same to detect the positive plateau in the second experiment, we can start by applying the low pass filter to the magnitude signal and then calculate the derivate.

In [ ]:
# Application of the low pass filter using a cut-off frequency of 10 Hz. The use_filtfilt argument states that
# the same filter is applied forward and backwards in order to reduce the noise the first filter may introduce.
filtered_magnitude_2 = bsnb.lowpass(magnitude_2, 10, fs=fs, use_filtfilt=True)

# Here we get the derivate of the signal of magnitude of the second experiment
derivate_magnitude_2 = np.diff(filtered_magnitude_2)

Now we have everything we need to apply a very similar algorithm as the one used to detect the negative plateau. Notice that the major difference on the next code cell are the conditions used to detect the limiting points. We also changed the names of the variables in order to clearly distinguish them from the previous case. In this case we do not explain each step of the algorithm because the steps are the same as on the previous case.

In [ ]:
# Here we initialise the list that will contain the provisory points limiting the plateau(s).
high_plateau = []

# Here we initialise the list that will contain the final points limiting the plateau(s).
high_final_plateau = []

# This flag will be used to tell us if the algorithm has found a left limiting point, because it does not make sense to look for consecutive
# left limiting points if we have not found a right one.
high_flag = False


###############################################################################
########################  Finding the limiting points  ########################
###############################################################################


# Iterate over all points in the derivate except the first and last ones. This is because we will be looking for left and right points, and, so,
# we cannot look to the left of the first point and the right of the last point.
for point in range(1, len(derivate_magnitude_2)-1):
    
    # Get the point imidiatelly at the left of the current one.
    left_point = derivate_magnitude_2[point-1]
    
    # Get the point imidiatelly at the right of the current one.
    right_point = derivate_magnitude_2[point+1]
    
    # This says that only if the flag is false, meaning that no left limiting point was found, does the algorithm look for a left limiting point.
    # Else, if it found a left limiting point, it will look for a right limiting point.
    if not high_flag:
        
        # Definition of point number 1
        # If all the identified conditions verify, we store the point as a left limiting point.
        if np.abs(left_point) > very_high_limiting_threshold and  left_point > 0 and np.abs(right_point) < very_low_limiting_threshold:
            high_plateau.append(point)
            
            # The flag turns to true to specify that we have found a left limiting point.
            high_flag = True
    else:
        # Definition of point number 3
        # We check if the time threshold specified before is respected and only if it is does the algorithm look for a right limiting point.
        if point - high_plateau[-1] > delta_t:
            
            # If all the identified conditions verify, we store the point as a right limiting point.
            if np.abs(left_point) < very_low_limiting_threshold and np.abs(right_point) > very_high_limiting_threshold and right_point < 0:
                high_plateau.append(point)
                
                # Then we say that the flag is false so that new left limiting points can be found.
                high_flag = False

# If a left limiting point was found but not the right corresponding one, we should discard it.
if high_flag:
    high_plateau.pop()

# Only if we have more than two points do we check if they are suitable to be considered as limits of a plateau.
if len(high_plateau) >= 2:
    
    # Here we will iterate over all points but in a pairwise condition, because a plateau is always limited by two points in our algorithm.
    for point in range(0, len(high_plateau)-2, 2):
        
        # Definition of point number 2
        
        # If there are no point in between limiting points with a derivate higher than the threshold, then those are really limiting points and
        # are stored in the final list.
        if not np.any(derivate_magnitude_2[high_plateau[point]:high_plateau[point+1]] > very_low_threshold):
            high_final_plateau.append(high_plateau[point])
            high_final_plateau.append(high_plateau[point+1])

Once again, we will check if the algorithm worked as expected and compare the estimated time using this algorithm to the time we calculated using the force platform signals. The plot below shows the points the algorithm found as the vertical lines and we see that the plateau was found.

In [ ]:
time_2 = times_2[0]

start_2 = (np.array(high_plateau) + begin_2)/fs

print(f"The estimated time is {np.diff(high_plateau)[0]/fs} which differs from the ground truth by {0.404 - np.diff(high_plateau)[0]/fs:.3}.")

bsnb.plot(times_2[0], filtered_magnitude_2, vert_lines=start_2)

In this <span class="color4"><strong>Jupyter Notebook</strong></span> we will use data from two different experiments due to the high variability between the signals acquired in the same conditions. So, we will load the signals of the second experiment in the next cell.

<table style="width:12%">
  <tr>
    <td><img src="../../images/extract/loft_time/loft_time_rui.gif" height="326" controls></td>
  </tr>
</table>

In [ ]:
# Define the path of the file that has the signals of each device
path_subject_2 = "../../signal_samples/loft_time_subject_2.h5"

# Load the data from the file
data_2 = bsnb.load(path_subject_2)

# Identify the mac addresses of each device
force_platform_2, accelerometer_2 = data_2.keys()

# Get the signals acquired by the force_platform
force_platform_signals_2 = data_2[force_platform_2]

# Get the signals acquired by the accelerometer
accelerometer_signals_2 = data_2[accelerometer_2]

<strong><span class="color1">The next cell will show how to get the data from each device and from each channel of each device.</span></strong>
<br> For that, we will first initiate different lists for different devices and different experiments. Then, we will iterate over each channel of each device and store those signals in the corresponding list.

In [ ]:
# These are the lists where we will store the signals from each channels of the force platform of each experiment.
# They are initiated as empty lists so we can append items to them.
force_platform_channels_2 = []

# And these are the lists where we will store the signals from each direction of the accelerometer of each experiment.
accelerometer_direction_2 = []


# This is the part where we get the signal from each channel and append it to the corresponding list.
# The syntax of it may be a bit odd at first sight, so we recommend you to read: https://realpython.com/iterate-through-dictionary-python/
# Basically, force_platform_signals_1.keys() is a strutucture that iterates over the channels of the force platform device.
for channel in force_platform_signals_2.keys():
    channel_signal_2 = force_platform_signals_2[channel]
    force_platform_channels_2.append(channel_signal_2)
    
# Now we need to do the same for the accelerometer. Note that the code is very similar compared to the force platform.
for direction in accelerometer_signals_2.keys():
    direction_signal_2 = accelerometer_signals_2[direction]
    accelerometer_direction_2.append(direction_signal_2)

The next plot shows the same data regarding the second experiment. We can see that force platform signals are very similar and can be directly analyzed, considering that the loft time corresponds to the time that no force is being applied to the platform. 

In [ ]:
fs = 500
time_2 = bsnb.generate_time(force_platform_signals_2['CH1'], fs)

begin_2 = 25*fs
end_2 = 30*fs

times_2 = []
force_platform_channels_2 = []
for key in force_platform_signals_2.keys():
    times_2.append(time_2[begin_2:end_2])
    force_platform_channels_2.append(force_platform_signals_2[key][begin_2:end_2])

tr_2 = np.min(force_platform_channels_2[0]) + np.ptp(force_platform_channels_2[0])*.01
index_2 = np.where(force_platform_channels_2[0] < tr_2)[0]
index_2 = (np.array([index_2[a] for a in [0, -1]]) + begin_2)/fs

bsnb.plot(times_2, force_platform_channels_2, grid_plot=True, grid_lines=2, grid_columns=2, x_axis_label='Time (s)', 
          vert_lines=[index_2, index_2, index_2, index_2],
          y_axis_label='Raw Data', title=["Channel 1", "Channel 2", "Channel 3", "Channel 4"])

We saw that accelerometer signals are not the most reliable to calculate the time of flight of a jumping person, but can still be used if a proper post-analysis is applied. Specifically, the signals should be analyzed before starting the development of any algorithm, in order to try to identify the most distinguishable structures that correspond to the actions we look for.

Furthermore, accelerometers may be a well suited solution for applications where high precision is not a concern, because they may not be as precise as other specific sensors. The big advantage of accelerometers is that they can be used outside of the laboratory and in a wide range of applications.

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<hr>
<table width="100%">
    <tr>
        <td style="border-right:solid 3px #009EE3" width="20%">
            <img src="../../images/ost_logo.png">
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/aux_files/biosignalsnotebooks_presentation.pdf" target="_blank">&#9740; Project Presentation</a>
            <br>
            <a href="https://github.com/biosignalsnotebooks/biosignalsnotebooks" target="_blank">&#9740; GitHub Repository</a>
            <br>
            <a href="https://pypi.org/project/biosignalsnotebooks/" target="_blank">&#9740; How to install biosignalsnotebooks Python package ?</a>
            <br>
            <a href="../MainFiles/signal_samples.ipynb">&#9740; Signal Library</a>
        </td>
        <td width="40%" style="text-align:left">
            <a href="../MainFiles/biosignalsnotebooks.ipynb">&#9740; Notebook Categories</a>
            <br>
            <a href="../MainFiles/by_diff.ipynb">&#9740; Notebooks by Difficulty</a>
            <br>
            <a href="../MainFiles/by_signal_type.ipynb">&#9740; Notebooks by Signal Type</a>
            <br>
            <a href="../MainFiles/by_tag.ipynb">&#9740; Notebooks by Tag</a>
        </td>
    </tr>
</table>

<span class="color6">**Auxiliary Code Segment (should not be replicated by
the user)**</span>

In [8]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'],
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>